In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
import zipfile
extracted = './training'
with zipfile.ZipFile('../input/dogs-vs-cats/train.zip', 'r') as zip_ref:
    zip_ref.extractall(extracted)

In [ ]:
extracted = './test'
with zipfile.ZipFile('../input/dogs-vs-cats/test1.zip', 'r') as zip_ref:
    zip_ref.extractall(extracted)

In [ ]:
TEST_PATH = './test/test1'
TRAIN_PATH = './training/train'
IMAGE_SIZE = 128

In [ ]:
TEST_PATH

In [ ]:
cat_paths = [os.path.join(TRAIN_PATH, file) for file in os.listdir(TRAIN_PATH) if 'cat' in file]
dog_paths = [os.path.join(TRAIN_PATH, file) for file in os.listdir(TRAIN_PATH) if 'dog' in file]

In [ ]:
def generator(filenames, labels):
    def callable_generator():
        for filename, lab in zip(filenames, labels):
            img = img_to_array(load_img(filename, target_size=(IMAGE_SIZE, IMAGE_SIZE)))/255.
            yield img, lab         
    return callable_generator

In [ ]:
labels = len(cat_paths)*[0] + len(dog_paths)*[1]
data = cat_paths+dog_paths

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data, labels, shuffle=True, test_size=0.2, random_state=42)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(generator(X_train, y_train),
                                              (tf.float32, tf.float32),
                                              ((IMAGE_SIZE, IMAGE_SIZE, 3), ()))

val_dataset = tf.data.Dataset.from_generator(generator(X_val, y_val),
                                              (tf.float32, tf.float32),
                                              ((IMAGE_SIZE, IMAGE_SIZE, 3), ()))

BATCH_SIZE = 4
def prep_data(train_dataset):
    train_dataset = train_dataset.cache().batch(BATCH_SIZE).repeat()
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return train_dataset

train_dataset = prep_data(train_dataset)

val_dataset = prep_data(val_dataset)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(tf.keras.layers.Conv2D(16, 3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.Conv2D(32, 3, activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy', metrics=['acc'], optimizer='adam')

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, validation_steps=len(X_val)//(BATCH_SIZE),
                    epochs=10, steps_per_epoch=len(X_train)//(BATCH_SIZE))

In [ ]:

import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()